In [ ]:
from src.collector import *
from src.combiner import *
from src.composer import *
from src.decomposer import *
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA
import numpy as np
import json

In [ ]:
def make_chart(years):
    # df = pd.read_csv(f"./cleaned/{year}_votes.csv")
    # df_meta = pd.read_csv(f"./cleaned/{year}_meta.csv")
    
    raw_data = Collector.collect_data(years, "h")
    json.dump(raw_data, open("./dump/raw_data.json", "w"))
    df, df_meta = Combiner.combine_all_roll_calls_into_df(raw_data)
    
    df = Composer.map_values(df)
    df = Composer.filter_excessive_abstentions(df)
    df = Composer.normalize_votes(df)
    
    df_meta = df_meta[df_meta["id"].isin(df["id"])].drop_duplicates("id")
    df_meta["color"] = np.where(df_meta["party"] == "D", "blue", "red")
    df_meta["label"] = df_meta["name"] + " (" + df_meta["party"] + "-" + df_meta["state"] + ")"
    
    mat = Composer.create_voting_matrix(df)
    
    u,s,v =  Decomposer.svd(mat)
    us = u * s
    np.savetxt("./dump/v.txt",
               v,
               fmt="%.3f",
               delimiter="\t"
    )
    

    fig = go.Figure(data=go.Scatter(
        x=us[:, 0], 
        y=us[:,1],
        mode="markers",
        marker=dict(color=df_meta["color"], size=5),
        text=df_meta["label"],
        hoverinfo="text",)
                )
    
    fig.update_layout(
        title=dict(
            text=str(years),
            x=0.5,
            xanchor='center',
            font=dict(size=20)
            
        ),
    )
    
    return fig


In [3]:
fig = make_chart([2024])
fig.show()

Found 517 for the year 2024
Found in total 517 entries


100%|██████████| 517/517 [00:01<00:00, 513.83it/s]
d:\projects\house\src\composer.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return df.replace(value_map)
